In [1]:
import pandas as pd
from megadetector.detection.run_detector_batch import load_and_run_detector_batch

In [2]:
batch_size = 20
images_paths = pd.read_csv('y_clean_thin.csv', index_col=0)['image_path']

In [ ]:
results = load_and_run_detector_batch(
       "MDV5A", images_paths, confidence_threshold=0.1, batch_size=20, n_cores=6)

Model v5a.0.1 already exists and is valid at /tmp/megadetector_models/md_v5a.0.1.pt
PyTorch reports 1 available CUDA devices
GPU available: True
PyTorch reports 1 available CUDA devices
GPU available: True
Bypassing imports for model type yolov5
Loading PT detector with compatibility mode classic
Loaded image size 1280 from model metadata
Using model stride: 64
PTDetector using device cuda:0


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


Loaded model in 0.42 seconds


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


In [10]:
images_paths_sec, categories, confs, bboxes, n_animals = [], [], [], [], []

for result in results:
    images_paths_sec.append(result.get('file'))
    
    detections = result.get('detections')
    if detections:
        detection = max(detections, key=lambda d: d["conf"])
        categories.append(detection.get('category'))
        confs.append(detection.get('conf'))
        bboxes.append(detection.get('bbox'))
        n_animals.append(len(detections))
    else:
        categories.append(None)
        confs.append(None)
        bboxes.append(None)
        n_animals.append(0)

results_df = pd.DataFrame({'image_path': images_paths_sec, 'category': categories, 'conf': confs, 'bbox': bboxes, 'n_animals': n_animals})

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%Y_%m_%d_%H_%M')
results_df.to_csv(f'megadetector_results_{now}.csv')

#### etc

In [ ]:
# for start in range(0, len(images_paths), batch_size):
#     end = start + batch_size
#     batch_paths = images_paths[start:end]
#     batch_results = load_and_run_detector_batch(
#        "MDV5A", images_paths, confidence_threshold=0.1, batch_size=20)
#     for result in batch_results:
#         images_paths_sec.append(result.get('file'))
#         categories.append(result.get('category'))
#         confs.append(result.get('conf'))
#         bboxes.append(result.get('bbox'))